In [2]:
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize,PorterStemmer
import re
import string



### Reading Data

In [3]:
hotels = pd.read_csv('Hotels_features_dataset.csv')


In [4]:
hotels.head()

,name,country,street,region,rating,reviews,amenities,rooms,types,price,official_description
0,WH Hotel,Lebanon,Lyon Street Hamra Emille Edde,NaN,4.0,"{'Excellent': 104, 'Good': 102, 'Average': 51,...","Free High Speed Internet (WiFi),Free breakfast...","Air conditioning,Fireplace,Housekeeping,Room s...","Non-smoking rooms,Suites,Family rooms,Smoking ...",NaN,NaN
1,Le Patio Boutique Hotel,Lebanon,"1144 Marfaa, Uruguay Street Solidere",NaN,4.5,"{'Excellent': 213, 'Good': 78, 'Average': 31, ...","Paid private parking nearby,Free High Speed In...","Air conditioning,Housekeeping,Room service,Saf...","Non-smoking rooms,Suites,Family rooms,Smoking ...",121.0,NaN
2,Riviera Hotel Beirut,Lebanon,"Avenue De Paris, Corniche El Manara Riad El Solh",NaN,3.5,"{'Excellent': 90, 'Good': 101, 'Average': 69, ...","Valet parking,Free High Speed Internet (WiFi),...","Air conditioning,Private balcony,Room service,...","Ocean view,Pool view,Non-smoking rooms,Suites,...",NaN,NaN
3,Le Bristol Beyrouth,Lebanon,Mme Curie Street Verdun,NaN,4.5,"{'Excellent': 219, 'Good': 79, 'Average': 25, ...","Free High Speed Internet (WiFi),Pool,Fitness C...","Air conditioning,Room service,Safe,VIP room fa...","Non-smoking rooms,Suites,Family rooms,Smoking ...",NaN,NaN
4,Golden Tulip Midtown Hotel And Suites,Lebanon,"Building Number 483, Sidani / Souraty Street M...",NaN,4.0,"{'Excellent': 147, 'Good': 86, 'Average': 32, ...","Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Safe,Kitchenette...","Non-smoking rooms,Suites,Family rooms,Smoking ...",NaN,NaN


In [4]:
features = set()

In [5]:
hotels['amenities'].apply(lambda item: [features.add(x) for x in str(item).split(',')])
hotels['rooms'].apply(lambda item: [features.add(x) for x in str(item).split(',')])

0        [None, None, None, None, None, None, None, Non...
1        [None, None, None, None, None, None, None, Non...
2        [None, None, None, None, None, None, None, Non...
3        [None, None, None, None, None, None, None, Non...
4        [None, None, None, None, None, None, None, Non...
                               ...                        
70102    [None, None, None, None, None, None, None, Non...
70103    [None, None, None, None, None, None, None, Non...
70104           [None, None, None, None, None, None, None]
70105    [None, None, None, None, None, None, None, Non...
70106    [None, None, None, None, None, None, None, Non...
Name: rooms, Length: 70107, dtype: object

In [6]:
features

{' DVDs',
 ' music for children',
 '24-hour check-in',
 '24-hour front desk',
 '24-hour security',
 'ATM on site',
 'Additional bathroom',
 'Adult pool',
 'Adults only',
 'Aerobics',
 'Aerobics offsite',
 'Air conditioning',
 'Air purifier',
 'Airport transportation',
 'Allergy-free room',
 'Archery',
 'Archery offsite',
 'BBQ facilities',
 'Babysitting',
 'Badminton',
 'Baggage storage',
 'Banquet room',
 'Bar / lounge',
 'Bath / shower',
 'Bathrobes',
 'Beach',
 'Bicycle rental',
 'Bicycle tours',
 'Bicycles available',
 'Bidet',
 'Billiards',
 'Blackout curtains',
 'Board games / puzzles',
 'Boating',
 'Body wrap',
 'Books',
 'Bottled water',
 'Bowling',
 'Bowling offsite',
 'Breakfast available',
 'Breakfast buffet',
 'Breakfast in the room',
 'Bridal suite',
 'Business Center with Internet Access',
 'Butler service',
 'Cable / satellite TV',
 'Canoeing',
 'Canoeing offsite',
 'Car hire',
 'Casino and Gambling',
 'Chapel / shrine',
 'Children Activities (Kid / Family Friendly)',
 "

### Feature Extraction

In [7]:
features = {x for x in features if pd.notna(x) or x != ''}

In [8]:
features.discard('')
features.discard('nan')
# features.discard(' DVDs')
# features.discard(' music for children')

In [9]:
features

{' DVDs',
 ' music for children',
 '24-hour check-in',
 '24-hour front desk',
 '24-hour security',
 'ATM on site',
 'Additional bathroom',
 'Adult pool',
 'Adults only',
 'Aerobics',
 'Aerobics offsite',
 'Air conditioning',
 'Air purifier',
 'Airport transportation',
 'Allergy-free room',
 'Archery',
 'Archery offsite',
 'BBQ facilities',
 'Babysitting',
 'Badminton',
 'Baggage storage',
 'Banquet room',
 'Bar / lounge',
 'Bath / shower',
 'Bathrobes',
 'Beach',
 'Bicycle rental',
 'Bicycle tours',
 'Bicycles available',
 'Bidet',
 'Billiards',
 'Blackout curtains',
 'Board games / puzzles',
 'Boating',
 'Body wrap',
 'Books',
 'Bottled water',
 'Bowling',
 'Bowling offsite',
 'Breakfast available',
 'Breakfast buffet',
 'Breakfast in the room',
 'Bridal suite',
 'Business Center with Internet Access',
 'Butler service',
 'Cable / satellite TV',
 'Canoeing',
 'Canoeing offsite',
 'Car hire',
 'Casino and Gambling',
 'Chapel / shrine',
 'Children Activities (Kid / Family Friendly)',
 "

In [10]:
features_df = pd.DataFrame(columns=features)

In [11]:
features_df

,hotel_amenity_kids_pool,hotel_amenity_internet,Photo copier / fax In business center,Paid private parking on-site,Hammam,hotel_amenity_pool_beach_towels,Hand massage,hotel_amenity_special_diet_menus,Car hire,Heated pool,...,Bidet,Walking tours,hotel_amenity_full_body_massage,Neck massage,hotel_amenity_neck_massage,Diving,Sun terrace,hotel_amenity_dining_area,Full body massage,hotel_amenity_separate_dining_area


### Getting each hotel feature as a Vector

In [12]:
amenities  = hotels['amenities'] + ',' +  hotels['rooms']
hotels_names = hotels['name']
col_names = features_df.columns
hotel_features = []
for i,amenity in enumerate(amenities):
    if pd.notna(amenity):
        feats = amenity.split(',')
        row = np.zeros(len(features))
        for feat in feats:
            if feat != '':
                index = np.where(col_names == feat)[0][0] #index
                row[index] = 1
        hotel_features.append(row)

In [96]:
features_df = pd.DataFrame(hotel_features,columns= features)
features_df.head()

,hotel_amenity_kids_pool,hotel_amenity_internet,Photo copier / fax In business center,Paid private parking on-site,Hammam,hotel_amenity_pool_beach_towels,Hand massage,hotel_amenity_special_diet_menus,Car hire,Heated pool,...,Bidet,Walking tours,hotel_amenity_full_body_massage,Neck massage,hotel_amenity_neck_massage,Diving,Sun terrace,hotel_amenity_dining_area,Full body massage,hotel_amenity_separate_dining_area
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
# features_df['rating'] = hotels['rating']

In [98]:
# features_df['rating']

In [99]:
features_df = features_df.rename(index=hotels_names)

### KNN Content-Based Recommender

In [100]:
from sklearn.neighbors import NearestNeighbors

In [101]:
knn = NearestNeighbors(n_neighbors=20,metric='cosine')

In [102]:
knn.fit(features_df)

NearestNeighbors(metric='cosine', n_neighbors=20)

In [103]:
distances,indecies = knn.kneighbors(features_df)

In [104]:
def get_recommendation(name):
    index = np.where(features_df.index == name)
    neighbours =  indecies[index].tolist()[0][1:21]
    for neighbour in neighbours:
            print(hotels['name'].loc[neighbour])

In [105]:
get_recommendation('WH Hotel')

Hampton Inn Suites Bolingbrook
Hotel Pamphylia
Brit Hotel Les Comtes de Champagne
DoubleTree by Hilton Istanbul Umraniye
Anadolu Hotels Downtown Ankara
Riccarton mall motel
Warwick Stone 55 Hotel
Hostel Good Luck
Wine Hostel
Maison de Marquay
Besakih Beach Hotel
Saifi Suites
Arion Palace Hotel
Vila Mila
Freehand New York
Old Moshi Hostel
Twinkle Valley
ibis budget Ambassador Seoul Dongdaemun
Altinel Ankara Hotel & Convention Center


### Testing Cosine Similarity for two Hotels

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm

In [57]:
def similarity(one,two):
    one = features_df.loc[one]
    two = features_df.loc[two]
    return np.dot(one,two)/ np.multiply(norm(one),norm(two))

In [58]:
similarity("Hampton Inn Suites Bolingbrook","Hotel Pamphylia")

0.7877263614433762

In [ ]:
get_recommendation("Blanc Inn")

In [ ]:
similarity("Blanc Inn","Athens Green Apartments")